<a href="https://colab.research.google.com/github/eschill04/decoupled-style-descriptors-eb/blob/ui/UI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title
%%capture
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/MyDrive/Colab Notebooks/Spring22/decoupled-style-descriptors-eb"

!pip install virtualenv
# !virtualenv .venv
!source venv/bin/activate
!pip install -r requirements.txt
!pip install gradio
!pip install tensorboardX
!pip install importlib
!pip install ffmpeg-python

In [ ]:
#@title
import torch
import argparse
import numpy as np
from helper import *
from config.GlobalVariables import *
from SynthesisNetwork import SynthesisNetwork
from DataLoader import DataLoader
import convenience
import gradio as gr

In [ ]:
#@title
from os import getpriority
device = 'cpu'
num_samples = 10

net = SynthesisNetwork(weight_dim=256, num_layers=3).to(device)

if not torch.cuda.is_available():
    try: # retrained model also contains loss in dict 
        net.load_state_dict(torch.load('./model/250000.pt', map_location=torch.device(device))["model_state_dict"])
    except:
        net.load_state_dict(torch.load('./model/250000.pt', map_location=torch.device(device)))
    

dl = DataLoader(num_writer=1, num_samples=10, divider=5.0, datadir='./data/writers')


writer_options = [80, 120] # find more writers to make more interesting
all_loaded_data = []
avail_char = "0 1 2 3 4 5 6 7 8 9 a b c d e f g h i j k l m n o p q r s t u v w x y z A B C D E F G H I J K L M N O P Q R S T U V W X Y Z ! ? \" ' * + - = : ; , . < > \ / [ ] ( ) # $ % &"
avail_char_list = avail_char.split(" ")
for writer_id in [80, 120]:
    loaded_data = dl.next_batch(TYPE='TRAIN', uid=writer_id, tids=list(range(num_samples)))
    all_loaded_data.append(loaded_data)

def interpolate_writers(target_word, val):
    image = convenience.sample_blended_writers([val, 1 - val], target_word, net, all_loaded_data, device).convert("RGB")
    return image

def choose_writers(writer1, writer2):
    writer_options = [int(writer1.split(" ")[1]), int(writer2.split(" ")[1])]
    all_loaded_data.clear()

    for writer_id in writer_options:
        loaded_data = dl.next_batch(TYPE='TRAIN', uid=writer_id, tids=list(range(num_samples)))
        all_loaded_data.append(loaded_data)

    return gr.Slider.update(label=f"Ratio of {writer1} vs. {writer2}")
'''
def choose_writer(writ, c1, c2, c3, c4, val):
    all_loaded_data.clear()
    w = int(writ.split(" ")[1])
    loaded_data = dl.next_batch(TYPE='TRAIN', uid=w, tids=list(range(num_samples)))
    all_loaded_data.append(loaded_data)
    return char_grid(c1, c2, c3, c4, val)
'''
def update_char(c1, c2, c3, c4):
      return gr.Button.update(value=f"Blend {c1}, {c2}, {c3}, and {c4}!")

def char_grid(c1, c2, c3, c4, val):
    image = convenience.sample_character_grid([c1, c2, c3, c4], val, net, all_loaded_data, device).convert("RGB")
    return image
    
with gr.Blocks() as demo:
    with gr.Tabs():
      with gr.TabItem("Blend Writers"):
        target_word = gr.Textbox(label="Target Word", value="hello world", max_lines=1)
        with gr.Row():
            radio_options = ["Writer " + str(n) for n in writer_options]
            with gr.Column():
                writer1 = gr.Radio(radio_options, value="Writer 80", label="Style for writer 1")
            with gr.Column():
                writer2 = gr.Radio(radio_options, value="Writer 120", label="Style for writer 2")
        slider = gr.Slider(0, 1, value=0.3, label="Ratio of Writer 80 vs. Writer 120")

        default_image = convenience.sample_blended_writers([0.3, 0.7], "hello world", net, all_loaded_data, device).convert("RGB")
        output = gr.Image(default_image)
        
        slider.change(fn=interpolate_writers, inputs=[target_word, slider], outputs=[output])
        target_word.submit(fn=interpolate_writers, inputs=[target_word, slider], outputs=[output])

        writer1.change(fn=choose_writers, inputs=[writer1, writer2], outputs=[slider])
        writer2.change(fn=choose_writers, inputs=[writer1, writer2], outputs=[slider])

      with gr.TabItem("Blend Characters"):
        with gr.Row():
            with gr.Column():
                char1 = gr.Dropdown(choices=avail_char_list, value="y", label="Character 1")
            with gr.Column():
                char2 = gr.Dropdown(choices=avail_char_list, value="s", label="Character 2")
            with gr.Column():
                char3 = gr.Dropdown(choices=avail_char_list, value="u", label="Character 3")
            with gr.Column():
                char4 = gr.Dropdown(choices=avail_char_list, value="n", label="Character 4")
        with gr.Row(): 
            submit_button = gr.Button(value="Blend y, s, u, and n!")
        '''
        with gr.Row():
            radio_options2 = ["Writer " + str(n) for n in writer_options]
            writer = gr.Radio(radio_options2, value="Writer 80", label="Style for Writer")
            writer.change(fn=choose_writer, inputs=[writer, char1, char2, char3, char4, slider2], outputs=[output])
        '''
        slider2 = gr.Slider(2, 20, value=10, label="Grid Size", step=1)

        default_image = convenience.sample_character_grid(['y', 's', 'u', 'n'], 10, net, all_loaded_data, device).convert("RGB")
        output = gr.Image(default_image)
        
        char1.change(fn=update_char, inputs=[char1, char2, char3, char4], outputs=[submit_button])
        char2.change(fn=update_char, inputs=[char1, char2, char3, char4], outputs=[submit_button])
        char3.change(fn=update_char, inputs=[char1, char2, char3, char4], outputs=[submit_button])
        char4.change(fn=update_char, inputs=[char1, char2, char3, char4], outputs=[submit_button])
        
        
        slider2.change(fn=char_grid, inputs=[char1, char2, char3, char4, slider2], outputs=[output])
        submit_button.click(fn=char_grid, inputs=[char1, char2, char3, char4, slider2], outputs=[output])
        
        

demo.launch()

self.datadir :  ./data/writers
Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://17255.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces: https://huggingface.co/spaces


(<gradio.routes.App at 0x7fd879c935d0>,
 'http://127.0.0.1:7871/',
 'https://17255.gradio.app')